# DATA PROCESSING FOR SUB DATA

In [1]:
# Importation des packages

# Base
import importlib
import functions
importlib.reload(functions)
from functions import *

# Data Management
import pandas as pd
import numpy as np

# Imput
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


/Users/hugopuybareau/Documents/GitHub/QRT_DATA_CHALLENGE/env_qrt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importation des données

home_teams = pd.read_csv('../data/Sub_Data/sub_home_team_statistics_df.csv', index_col=0)
away_teams = pd.read_csv('../data/Sub_Data/sub_away_team_statistics_df.csv', index_col=0)

In [3]:
# Renommer les colonnes pour pouvoir les différencier une fois les deux DF aggrégés.

home_teams.columns = 'HOME_' + home_teams.columns
away_teams.columns = 'AWAY_' + away_teams.columns

# Faire la concaténation
sub_data_teams = pd.concat([home_teams,away_teams],join='inner',axis=1)

sub_data_teams.shape

(25368, 280)

In [4]:
# Traitement des valeurs manquantes pour les statistiques sur les équipes

# nan_colormap(sub_data_teams, 'Vérification présence de NaNs')
spot_columns(sub_data_teams, percent=10)

Column 'HOME_TEAM_INJURIES_5_last_match_sum' has 10.66% NaN values
Column 'HOME_TEAM_INJURIES_5_last_match_average' has 10.64% NaN values
Column 'HOME_TEAM_BALL_SAFE_5_last_match_std' has 11.40% NaN values
Column 'HOME_TEAM_INJURIES_5_last_match_std' has 14.33% NaN values
Column 'AWAY_TEAM_INJURIES_5_last_match_sum' has 10.61% NaN values
Column 'AWAY_TEAM_INJURIES_5_last_match_average' has 10.59% NaN values
Column 'AWAY_TEAM_BALL_SAFE_5_last_match_std' has 11.39% NaN values
Column 'AWAY_TEAM_INJURIES_5_last_match_std' has 14.49% NaN values


In [5]:
# Pour l'imputing, on utilise la même stratégie quand dans le data_processing_train
# sauf qu'on va remplacer les 20% par 10%. 

# Je remplis d'abord toutes les colonnes où on observe moins de 10% de NaNs.

for col in sub_data_teams.columns[sub_data_teams.isna().mean() < 0.2]:
    sub_data_teams.fillna({col: sub_data_teams[col].median()}, inplace=True)

spot_columns(sub_data_teams, percent=0)

In [6]:
# Maintenant je remplis le reste avec un KNN Imputer. 

# sub_data_teams = sub_data_teams.reset_index()

# imputer = KNNImputer(n_neighbors=5)
# sub_data_teams = pd.DataFrame(imputer.fit_transform(sub_data_teams), columns=sub_data_teams.columns)

# spot_columns(sub_data_teams, percent=0)

In [7]:
# sub_data_teams.head(15)

In [8]:
# Ajout des données 'joueurs' 

home_players = pd.read_csv('../data/Sub_Data/sub_home_player_statistics_df.csv')
away_players = pd.read_csv('../data/Sub_Data/sub_away_player_statistics_df.csv')

train_home_players = pd.read_csv('../data/Train_Data/home_players_df.csv')
train_away_players = pd.read_csv('../data/Train_Data/away_players_df.csv')

# Renommer les colonnes pour pouvoir les différencier une fois les deux DF aggrégés.

home_players.columns = 'HOME_' + home_players.columns
away_players.columns = 'AWAY_' + away_players.columns

home_players = replacing(home_players)
away_players = replacing(away_players)

# home_players.shape --> (504626, 304)
# away_players.shape --> (504626, 304)
# nan_colormap(home_players, 'color map pour les home players')

In [9]:
# Vérification colonnes supprimées dans les données d'entraînement

def find_missing_columns_bidirectional(df1: pd.DataFrame, df2: pd.DataFrame) -> dict:

    missing_in_df2 = [col for col in df1.columns if col not in df2.columns]
    missing_in_df1 = [col for col in df2.columns if col not in df1.columns]
    
    return {
        'missing_in_df2': missing_in_df2,
        'missing_in_df1': missing_in_df1
    }

missing_columns_home_1 = find_missing_columns_bidirectional(train_home_players, home_players)

# print("Colonnes présentes dans 1 mais manquantes dans 2 :", missing_columns['missing_in_df2'])
# print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns['missing_in_df1'])

home_players = home_players.drop(columns=missing_columns_home_1['missing_in_df1'])

missing_columns_home_2 = find_missing_columns_bidirectional(train_home_players, home_players)

print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns_home_2['missing_in_df1'])

Colonnes présentes dans 2 mais manquantes dans 1 : []


In [10]:
missing_columns_away_1 = find_missing_columns_bidirectional(train_away_players, away_players)

# print("Colonnes présentes dans 1 mais manquantes dans 2 :", missing_columns['missing_in_df2'])
# print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns['missing_in_df1'])

away_players = away_players.drop(columns=missing_columns_away_1['missing_in_df1'])

missing_columns_away_2 = find_missing_columns_bidirectional(train_away_players, away_players)

print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns_away_2['missing_in_df1'])

Colonnes présentes dans 2 mais manquantes dans 1 : []


In [11]:
# Pour le traitement des NaNs dans ces tables, on fait la même chose que pour les équipes.
# On pense quand même à enlever les colonnes qui ont été supprimées dans le traitement des données
# d'entraînement. 

In [12]:
# Je remplis ensuite toutes les colonnes où on observe moins de 20% de NaNs.

home_players = home_players.fillna({'HOME_POSITION': home_players['HOME_POSITION'].mode()[0]})

for col in home_players.columns[home_players.isna().mean() < 0.5]:
    if home_players[col].dtype in ['float64', 'int64']:
        home_players.fillna({col: home_players[col].median()}, inplace=True)


spot_columns(home_players, percent=0)

In [13]:
# Je remplis ensuite toutes les colonnes où on observe moins de 20% de NaNs.

away_players = away_players.fillna({'AWAY_POSITION': away_players['AWAY_POSITION'].mode()[0]})

for col in away_players.columns[away_players.isna().mean() < 0.5]:
    if away_players[col].dtype in ['float64', 'int64']:
        away_players.fillna({col: away_players[col].median()}, inplace=True)

spot_columns(away_players, percent=0)


In [14]:
# Maintenant je remplis le reste avec un KNN Imputer. 

# imputer_home = KNNImputer(n_neighbors=3)
# home_players_imputed = pd.DataFrame(
#     imputer_home.fit_transform(home_players.drop(columns=['HOME_POSITION'], axis=1)),
#     columns=home_players.columns.drop('HOME_POSITION')
# )

# home_players = pd.concat([home_players['HOME_POSITION'], home_players_imputed], axis=1)

# home_players.head(5)

In [15]:
# imputer_away = KNNImputer(n_neighbors=3)
# away_players_imputed = pd.DataFrame(
#     imputer_away.fit_transform(away_players.drop(columns=['AWAY_POSITION'], axis=1)),
#     columns=away_players.columns.drop('AWAY_POSITION')
# )

# away_players = pd.concat([away_players['AWAY_POSITION'], away_players_imputed], axis=1)

# away_players.head(5)

In [16]:
# Je remets les colonnes dans le bonne ordre même si ce n'est pas obligatoire.

# cols = list(home_players.columns)
# col1_idx, col2_idx = cols.index('HOME_POSITION'), cols.index('HOME_ID')
# cols[col1_idx], cols[col2_idx] = cols[col2_idx], cols[col1_idx]
# home_players = home_players[cols]

# cols = list(away_players.columns)
# col1_idx, col2_idx = cols.index('AWAY_POSITION'), cols.index('AWAY_ID')
# cols[col1_idx], cols[col2_idx] = cols[col2_idx], cols[col1_idx]
# away_players = away_players[cols]

In [17]:
# Je vais agréger les colonnes par poste et par matchs pour réduire la taile du dataframe des joueurs.
# Il est important de le faire avant la fusion pour garder un lien entre les joueurs et les équipes comme 
# TEAM_NAME n'est pas dans le dataframe de soumission. 

columns_to_aggregate_home = [col for col in home_players.columns if col.endswith(('sum', 'std'))]
columns_to_aggregate_away = [col for col in away_players.columns if col.endswith(('sum', 'std'))]

# len(columns_to_aggregate_home)
# On garde 98+2 colonnes.

In [18]:
# Création des tables home_agg et away_agg pour les agréger avec la table sub_data_teams.

home_agg = home_players.groupby(['HOME_ID', 'HOME_POSITION']).agg({
    col: 'mean' for col in columns_to_aggregate_home  # Agréger par la moyenne
}).reset_index()

away_agg = away_players.groupby(['AWAY_ID', 'AWAY_POSITION']).agg({
    col: 'mean' for col in columns_to_aggregate_away  # Agréger par la moyenne également
}).reset_index()

# home_agg.head(15)
away_agg.head(5)

,AWAY_ID,AWAY_POSITION,AWAY_PLAYER_ACCURATE_CROSSES_season_sum,AWAY_PLAYER_ACCURATE_PASSES_season_sum,AWAY_PLAYER_AERIALS_WON_season_sum,AWAY_PLAYER_ASSISTS_season_sum,AWAY_PLAYER_BIG_CHANCES_CREATED_season_sum,AWAY_PLAYER_BIG_CHANCES_MISSED_season_sum,AWAY_PLAYER_BLOCKED_SHOTS_season_sum,AWAY_PLAYER_CLEARANCES_season_sum,AWAY_PLAYER_CLEARANCE_OFFLINE_season_sum,AWAY_PLAYER_DISPOSSESSED_season_sum,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_sum,AWAY_PLAYER_DRIBBLED_PAST_season_sum,AWAY_PLAYER_DUELS_LOST_season_sum,AWAY_PLAYER_DUELS_WON_season_sum,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_sum,AWAY_PLAYER_FOULS_season_sum,AWAY_PLAYER_FOULS_DRAWN_season_sum,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum,AWAY_PLAYER_GOALS_season_sum,AWAY_PLAYER_GOALS_CONCEDED_season_sum,AWAY_PLAYER_HIT_WOODWORK_season_sum,AWAY_PLAYER_INTERCEPTIONS_season_sum,AWAY_PLAYER_KEY_PASSES_season_sum,AWAY_PLAYER_MINUTES_PLAYED_season_sum,AWAY_PLAYER_OFFSIDES_season_sum,AWAY_PLAYER_OWN_GOALS_season_sum,AWAY_PLAYER_PASSES_season_sum,AWAY_PLAYER_PENALTIES_COMMITTED_season_sum,AWAY_PLAYER_PENALTIES_MISSES_season_sum,AWAY_PLAYER_PENALTIES_SAVED_season_sum,AWAY_PLAYER_PENALTIES_SCORED_season_sum,AWAY_PLAYER_PENALTIES_WON_season_sum,AWAY_PLAYER_REDCARDS_season_sum,AWAY_PLAYER_SAVES_season_sum,AWAY_PLAYER_SAVES_INSIDE_BOX_season_sum,AWAY_PLAYER_SHOTS_BLOCKED_season_sum,AWAY_PLAYER_SHOTS_ON_TARGET_season_sum,AWAY_PLAYER_SHOTS_TOTAL_season_sum,AWAY_PLAYER_STARTING_LINEUP_season_sum,AWAY_PLAYER_SUCCESSFUL_DRIBBLES_season_sum,AWAY_PLAYER_TACKLES_season_sum,AWAY_PLAYER_TOTAL_CROSSES_season_sum,AWAY_PLAYER_TOTAL_DUELS_season_sum,AWAY_PLAYER_YELLOWCARDS_season_sum,AWAY_PLAYER_ACCURATE_CROSSES_season_std,AWAY_PLAYER_ACCURATE_PASSES_season_std,AWAY_PLAYER_ACCURATE_PASSES_PERCENTAGE_season_std,AWAY_PLAYER_AERIALS_WON_season_std,AWAY_PLAYER_ASSISTS_season_std,AWAY_PLAYER_BIG_CHANCES_CREATED_season_std,AWAY_PLAYER_BIG_CHANCES_MISSED_season_std,AWAY_PLAYER_BLOCKED_SHOTS_season_std,AWAY_PLAYER_CLEARANCES_season_std,AWAY_PLAYER_CLEARANCE_OFFLINE_season_std,AWAY_PLAYER_DISPOSSESSED_season_std,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_std,AWAY_PLAYER_DRIBBLED_PAST_season_std,AWAY_PLAYER_DUELS_LOST_season_std,AWAY_PLAYER_DUELS_WON_season_std,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_std,AWAY_PLAYER_FOULS_season_std,AWAY_PLAYER_FOULS_DRAWN_season_std,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_std,AWAY_PLAYER_GOALS_season_std,AWAY_PLAYER_GOALS_CONCEDED_season_std,AWAY_PLAYER_HIT_WOODWORK_season_std,AWAY_PLAYER_INTERCEPTIONS_season_std,AWAY_PLAYER_KEY_PASSES_season_std,AWAY_PLAYER_MINUTES_PLAYED_season_std,AWAY_PLAYER_OFFSIDES_season_std,AWAY_PLAYER_OWN_GOALS_season_std,AWAY_PLAYER_PASSES_season_std,AWAY_PLAYER_PENALTIES_COMMITTED_season_std,AWAY_PLAYER_PENALTIES_MISSES_season_std,AWAY_PLAYER_PENALTIES_SAVED_season_std,AWAY_PLAYER_PENALTIES_SCORED_season_std,AWAY_PLAYER_PENALTIES_WON_season_std,AWAY_PLAYER_RATING_season_std,AWAY_PLAYER_REDCARDS_season_std,AWAY_PLAYER_SAVES_season_std,AWAY_PLAYER_SAVES_INSIDE_BOX_season_std,AWAY_PLAYER_SHOTS_BLOCKED_season_std,AWAY_PLAYER_SHOTS_ON_TARGET_season_std,AWAY_PLAYER_SHOTS_TOTAL_season_std,AWAY_PLAYER_STARTING_LINEUP_season_std,AWAY_PLAYER_SUCCESSFUL_DRIBBLES_season_std,AWAY_PLAYER_TACKLES_season_std,AWAY_PLAYER_TOTAL_CROSSES_season_std,AWAY_PLAYER_TOTAL_DUELS_season_std,AWAY_PLAYER_YELLOWCARDS_season_std,AWAY_PLAYER_ACCURATE_CROSSES_5_last_match_sum,AWAY_PLAYER_ACCURATE_PASSES_5_last_match_sum,AWAY_PLAYER_AERIALS_WON_5_last_match_sum,AWAY_PLAYER_ASSISTS_5_last_match_sum,AWAY_PLAYER_BIG_CHANCES_CREATED_5_last_match_sum,AWAY_PLAYER_BIG_CHANCES_MISSED_5_last_match_sum,AWAY_PLAYER_BLOCKED_SHOTS_5_last_match_sum,AWAY_PLAYER_CLEARANCES_5_last_match_sum,AWAY_PLAYER_CLEARANCE_OFFLINE_5_last_match_sum,AWAY_PLAYER_DISPOSSESSED_5_last_match_sum,AWAY_PLAYER_DRIBBLED_ATTEMPTS_5_last_match_sum,AWAY_PLAYER_DRIBBLED_PAST_5_last_match_sum,AWAY_PLAYER_DUELS_LOST_5_last_match_sum,AWAY_PLAYER_DUELS_WON_5_last_match_sum,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_5_last_match_sum,AWAY_PLAYE

In [19]:
# Pivot pour home_agg
home_pivot = home_agg.pivot(index='HOME_ID', columns='HOME_POSITION', values=columns_to_aggregate_home)

home_pivot.head(5)

HOME_PLAYER_ACCURATE_CROSSES_season_sum                        \
HOME_POSITION                                attacker   defender goalkeeper   
HOME_ID                                                                       
12303                                        2.500000  14.500000        0.0   
12304                                        5.666667  10.500000        0.0   
12305                                        0.250000   1.250000        0.0   
12306                                             NaN   4.500000        0.0   
12307                                       10.250000  17.428571        0.0   

                         HOME_PLAYER_ACCURATE_PASSES_season_sum             \
HOME_POSITION midfielder                               attacker   defender   
HOME_ID                                                                      
12303           8.909091                              11.250000  35.000000   
12304           4.625000                              13.333333  30.333333   
12305          21.125000                               9.500000  47.250000   
12306           9.400000                                    NaN  26.166667   
12307           2.400000                               8.250000  19.571429   

                                    HOME_PLAYER_AERIALS_WON_season_sum  \
HOME_POSITION goalkeeper midfielder                           attacker   
HOME_ID                                                                  
12303               13.0  16.545455                           1.750000   
12304               14.0  11.500000                          21.666667   
12305               14.5  28.625000                          12.500000   
12306                9.5  29.300000                                NaN   
12307                6.0  20.600000                          18.250000   

                                               HOME_PLAYER_ASSISTS_season_sum  \
HOME_POSITION   defender goalkeeper midfielder                       attacker   
HOME_ID                                                                         
12303          16.000000        1.0   7.363636                       5.000000   
12304          24.000000        3.0   6.250000                      20.333333   
12305          22.750000        0.5   8.375000                      22.500000   
12306          28.333333        6.0  17.700000                            NaN   
12307          18.142857        0.5  14.800000                       9.250000   

                                                \
HOME_POSITION   defender goalkeeper midfielder   
HOME_ID                                          
12303           6.666667        0.0  10.909091   
12304           4.000000        0.0   4.500000   
12305           1.750000        3.5  28.750000   
12306           5.833333        3.5  19.800000   
12307          10.714286        0.0   0.000000   

              HOME_PLAYER_BIG_CHANCES_CREATED_season_sum             \
HOME_POSITION                                   attacker   defender   
HOME_ID                                                               
12303                                           7.000000   4.666667   
12304                                          21.333333  13.333333   
12305                                          16.000000   3.500000   
12306                                                NaN   7.500000   
12307                                          21.750000  19.428571   

                                    HOME_PLAYER_BIG_CHANCES_MISSED_season_sum  \
HOME_POSITION goalkeeper midfielder                                  attacker   
HOME_ID                                                                         
12303                0.0   7.636364                                      0.00   
12304                0.0   6.375000                                     23.00   
12305                0.0  28.250000                                     16.50   
12306                0.0  20.600000                       

In [20]:
# Aplatir les colonnes multi-niveaux générées par pivot_table
home_pivot.columns = ['_'.join(col).strip() for col in home_pivot.columns.values]
home_pivot.reset_index(inplace=True)

home_pivot.head(5)

# Tout fonctionne bien, il ne restera plus qu'à traiter les NaNs. 

,HOME_ID,HOME_PLAYER_ACCURATE_CROSSES_season_sum_attacker,HOME_PLAYER_ACCURATE_CROSSES_season_sum_defender,HOME_PLAYER_ACCURATE_CROSSES_season_sum_goalkeeper,HOME_PLAYER_ACCURATE_CROSSES_season_sum_midfielder,HOME_PLAYER_ACCURATE_PASSES_season_sum_attacker,HOME_PLAYER_ACCURATE_PASSES_season_sum_defender,HOME_PLAYER_ACCURATE_PASSES_season_sum_goalkeeper,HOME_PLAYER_ACCURATE_PASSES_season_sum_midfielder,HOME_PLAYER_AERIALS_WON_season_sum_attacker,HOME_PLAYER_AERIALS_WON_season_sum_defender,HOME_PLAYER_AERIALS_WON_season_sum_goalkeeper,HOME_PLAYER_AERIALS_WON_season_sum_midfielder,HOME_PLAYER_ASSISTS_season_sum_attacker,HOME_PLAYER_ASSISTS_season_sum_defender,HOME_PLAYER_ASSISTS_season_sum_goalkeeper,HOME_PLAYER_ASSISTS_season_sum_midfielder,HOME_PLAYER_BIG_CHANCES_CREATED_season_sum_attacker,HOME_PLAYER_BIG_CHANCES_CREATED_season_sum_defender,HOME_PLAYER_BIG_CHANCES_CREATED_season_sum_goalkeeper,HOME_PLAYER_BIG_CHANCES_CREATED_season_sum_midfielder,HOME_PLAYER_BIG_CHANCES_MISSED_season_sum_attacker,HOME_PLAYER_BIG_CHANCES_MISSED_season_sum_defender,HOME_PLAYER_BIG_CHANCES_MISSED_season_sum_goalkeeper,HOME_PLAYER_BIG_CHANCES_MISSED_season_sum_midfielder,HOME_PLAYER_BLOCKED_SHOTS_season_sum_attacker,HOME_PLAYER_BLOCKED_SHOTS_season_sum_defender,HOME_PLAYER_BLOCKED_SHOTS_season_sum_goalkeeper,HOME_PLAYER_BLOCKED_SHOTS_season_sum_midfielder,HOME_PLAYER_CLEARANCES_season_sum_attacker,HOME_PLAYER_CLEARANCES_season_sum_defender,HOME_PLAYER_CLEARANCES_season_sum_goalkeeper,HOME_PLAYER_CLEARANCES_season_sum_midfielder,HOME_PLAYER_CLEARANCE_OFFLINE_season_sum_attacker,HOME_PLAYER_CLEARANCE_OFFLINE_season_sum_defender,HOME_PLAYER_CLEARANCE_OFFLINE_season_sum_goalkeeper,HOME_PLAYER_CLEARANCE_OFFLINE_season_sum_midfielder,HOME_PLAYER_DISPOSSESSED_season_sum_attacker,HOME_PLAYER_DISPOSSESSED_season_sum_defender,HOME_PLAYER_DISPOSSESSED_season_sum_goalkeeper,HOME_PLAYER_DISPOSSESSED_season_sum_midfielder,HOME_PLAYER_DRIBBLED_ATTEMPTS_season_sum_attacker,HOME_PLAYER_DRIBBLED_ATTEMPTS_season_sum_defender,HOME_PLAYER_DRIBBLED_ATTEMPTS_season_sum_goalkeeper,HOME_PLAYER_DRIBBLED_ATTEMPTS_season_sum_midfielder,HOME_PLAYER_DRIBBLED_PAST_season_sum_attacker,HOME_PLAYER_DRIBBLED_PAST_season_sum_defender,HOME_PLAYER_DRIBBLED_PAST_season_sum_goalkeeper,HOME_PLAYER_DRIBBLED_PAST_season_sum_midfielder,HOME_PLAYER_DUELS_LOST_season_sum_attacker,HOME_PLAYER_DUELS_LOST_season_sum_defender,HOME_PLAYER_DUELS_LOST_season_sum_goalkeeper,HOME_PLAYER_DUELS_LOST_season_sum_midfielder,HOME_PLAYER_DUELS_WON_season_sum_attacker,HOME_PLAYER_DUELS_WON_season_sum_defender,HOME_PLAYER_DUELS_WON_season_sum_goalkeeper,HOME_PLAYER_DUELS_WON_season_sum_midfielder,HOME_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_attacker,HOME_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_defender,HOME_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_goalkeeper,HOME_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_midfielder,HOME_PLAYER_FOULS_season_sum_attacker,HOME_PLAYER_FOULS_season_sum_defender,HOME_PLAYER_FOULS_season_sum_goalkeeper,HOME_PLAYER_FOULS_season_sum_midfielder,HOME_PLAYER_FOULS_DRAWN_season_sum_attacker,HOME_PLAYER_FOULS_DRAWN_season_sum_defender,HOME_PLAYER_FOULS_DRAWN_season_sum_goalkeeper,HOME_PLAYER_FOULS_DRAWN_season_sum_midfielder,HOME_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_attacker,HOME_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_defender,HOME_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_goalkeeper,HOME_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_midfielder,HOME_PLAYER_GOALS_season_sum_attacker,HOME_PLAYER_GOALS_season_sum_defender,HOME_PLAYER_GOALS_season_sum_goalkeeper,HOME_PLAYER_GOALS_season_sum_midfielder,HOME_PLAYER_GOALS_CONCEDED_season_sum_attacker,HOME_PLAYER_GOALS_CONCEDED_season_sum_defender,HOME_PLAYER_GOALS_CONCEDED_season_sum_goalkeeper,HOME_PLAYER_GOALS_CONCEDED_season_sum_midfielder,HOME_PLAYER_HIT_WOODWORK_season_sum_attacker,HOME_PLAYER_HIT_WOODWORK_season_sum_defender,HOME_PLAYER_HIT_WOODWORK_season_sum_goalkeeper,HOME_PLAYER_HIT_WOODWORK_season_sum_midfielder,HOME_PLAYER_INTERCEPTIONS_s

In [21]:
# Pivot pour away_agg
away_pivot = away_agg.pivot(index='AWAY_ID', columns='AWAY_POSITION', values=columns_to_aggregate_away)

# Aplatir les colonnes multi-niveaux générées par pivot_table
away_pivot.columns = ['_'.join(col).strip() for col in away_pivot.columns.values]
away_pivot.reset_index(inplace=True)

away_pivot.head(5)

,AWAY_ID,AWAY_PLAYER_ACCURATE_CROSSES_season_sum_attacker,AWAY_PLAYER_ACCURATE_CROSSES_season_sum_defender,AWAY_PLAYER_ACCURATE_CROSSES_season_sum_goalkeeper,AWAY_PLAYER_ACCURATE_CROSSES_season_sum_midfielder,AWAY_PLAYER_ACCURATE_PASSES_season_sum_attacker,AWAY_PLAYER_ACCURATE_PASSES_season_sum_defender,AWAY_PLAYER_ACCURATE_PASSES_season_sum_goalkeeper,AWAY_PLAYER_ACCURATE_PASSES_season_sum_midfielder,AWAY_PLAYER_AERIALS_WON_season_sum_attacker,AWAY_PLAYER_AERIALS_WON_season_sum_defender,AWAY_PLAYER_AERIALS_WON_season_sum_goalkeeper,AWAY_PLAYER_AERIALS_WON_season_sum_midfielder,AWAY_PLAYER_ASSISTS_season_sum_attacker,AWAY_PLAYER_ASSISTS_season_sum_defender,AWAY_PLAYER_ASSISTS_season_sum_goalkeeper,AWAY_PLAYER_ASSISTS_season_sum_midfielder,AWAY_PLAYER_BIG_CHANCES_CREATED_season_sum_attacker,AWAY_PLAYER_BIG_CHANCES_CREATED_season_sum_defender,AWAY_PLAYER_BIG_CHANCES_CREATED_season_sum_goalkeeper,AWAY_PLAYER_BIG_CHANCES_CREATED_season_sum_midfielder,AWAY_PLAYER_BIG_CHANCES_MISSED_season_sum_attacker,AWAY_PLAYER_BIG_CHANCES_MISSED_season_sum_defender,AWAY_PLAYER_BIG_CHANCES_MISSED_season_sum_goalkeeper,AWAY_PLAYER_BIG_CHANCES_MISSED_season_sum_midfielder,AWAY_PLAYER_BLOCKED_SHOTS_season_sum_attacker,AWAY_PLAYER_BLOCKED_SHOTS_season_sum_defender,AWAY_PLAYER_BLOCKED_SHOTS_season_sum_goalkeeper,AWAY_PLAYER_BLOCKED_SHOTS_season_sum_midfielder,AWAY_PLAYER_CLEARANCES_season_sum_attacker,AWAY_PLAYER_CLEARANCES_season_sum_defender,AWAY_PLAYER_CLEARANCES_season_sum_goalkeeper,AWAY_PLAYER_CLEARANCES_season_sum_midfielder,AWAY_PLAYER_CLEARANCE_OFFLINE_season_sum_attacker,AWAY_PLAYER_CLEARANCE_OFFLINE_season_sum_defender,AWAY_PLAYER_CLEARANCE_OFFLINE_season_sum_goalkeeper,AWAY_PLAYER_CLEARANCE_OFFLINE_season_sum_midfielder,AWAY_PLAYER_DISPOSSESSED_season_sum_attacker,AWAY_PLAYER_DISPOSSESSED_season_sum_defender,AWAY_PLAYER_DISPOSSESSED_season_sum_goalkeeper,AWAY_PLAYER_DISPOSSESSED_season_sum_midfielder,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_sum_attacker,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_sum_defender,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_sum_goalkeeper,AWAY_PLAYER_DRIBBLED_ATTEMPTS_season_sum_midfielder,AWAY_PLAYER_DRIBBLED_PAST_season_sum_attacker,AWAY_PLAYER_DRIBBLED_PAST_season_sum_defender,AWAY_PLAYER_DRIBBLED_PAST_season_sum_goalkeeper,AWAY_PLAYER_DRIBBLED_PAST_season_sum_midfielder,AWAY_PLAYER_DUELS_LOST_season_sum_attacker,AWAY_PLAYER_DUELS_LOST_season_sum_defender,AWAY_PLAYER_DUELS_LOST_season_sum_goalkeeper,AWAY_PLAYER_DUELS_LOST_season_sum_midfielder,AWAY_PLAYER_DUELS_WON_season_sum_attacker,AWAY_PLAYER_DUELS_WON_season_sum_defender,AWAY_PLAYER_DUELS_WON_season_sum_goalkeeper,AWAY_PLAYER_DUELS_WON_season_sum_midfielder,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_attacker,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_defender,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_goalkeeper,AWAY_PLAYER_ERROR_LEAD_TO_GOAL_season_sum_midfielder,AWAY_PLAYER_FOULS_season_sum_attacker,AWAY_PLAYER_FOULS_season_sum_defender,AWAY_PLAYER_FOULS_season_sum_goalkeeper,AWAY_PLAYER_FOULS_season_sum_midfielder,AWAY_PLAYER_FOULS_DRAWN_season_sum_attacker,AWAY_PLAYER_FOULS_DRAWN_season_sum_defender,AWAY_PLAYER_FOULS_DRAWN_season_sum_goalkeeper,AWAY_PLAYER_FOULS_DRAWN_season_sum_midfielder,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_attacker,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_defender,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_goalkeeper,AWAY_PLAYER_GOALKEEPER_GOALS_CONCEDED_season_sum_midfielder,AWAY_PLAYER_GOALS_season_sum_attacker,AWAY_PLAYER_GOALS_season_sum_defender,AWAY_PLAYER_GOALS_season_sum_goalkeeper,AWAY_PLAYER_GOALS_season_sum_midfielder,AWAY_PLAYER_GOALS_CONCEDED_season_sum_attacker,AWAY_PLAYER_GOALS_CONCEDED_season_sum_defender,AWAY_PLAYER_GOALS_CONCEDED_season_sum_goalkeeper,AWAY_PLAYER_GOALS_CONCEDED_season_sum_midfielder,AWAY_PLAYER_HIT_WOODWORK_season_sum_attacker,AWAY_PLAYER_HIT_WOODWORK_season_sum_defender,AWAY_PLAYER_HIT_WOODWORK_season_sum_goalkeeper,AWAY_PLAYER_HIT_WOODWORK_season_sum_midfielder,AWAY_PLAYER_INTERCEPTIONS_s

In [22]:
# Je fais maintenant le merge sur l'ID des matchs. 

home_pivot.rename(columns={'HOME_ID': 'ID'}, inplace=True)
away_pivot.rename(columns={'AWAY_ID': 'ID'}, inplace=True)

sub_data_players = home_pivot.merge(away_pivot, on='ID', how='inner')

sub_data_players.head(5)

ID  HOME_PLAYER_ACCURATE_CROSSES_season_sum_attacker  \
0  12303                                          2.500000   
1  12304                                          5.666667   
2  12305                                          0.250000   
3  12306                                               NaN   
4  12307                                         10.250000   

   HOME_PLAYER_ACCURATE_CROSSES_season_sum_defender  \
0                                         14.500000   
1                                         10.500000   
2                                          1.250000   
3                                          4.500000   
4                                         17.428571   

   HOME_PLAYER_ACCURATE_CROSSES_season_sum_goalkeeper  \
0                                                0.0    
1                                                0.0    
2                                                0.0    
3                                                0.0    
4                                                0.0    

   HOME_PLAYER_ACCURATE_CROSSES_season_sum_midfielder  \
0                                           8.909091    
1                                           4.625000    
2                                          21.125000    
3                                           9.400000    
4                                           2.400000    

   HOME_PLAYER_ACCURATE_PASSES_season_sum_attacker  \
0                                        11.250000   
1                                        13.333333   
2                                         9.500000   
3                                              NaN   
4                                         8.250000   

   HOME_PLAYER_ACCURATE_PASSES_season_sum_defender  \
0                                        35.000000   
1                                        30.333333   
2                                        47.250000   
3                                        26.166667   
4                                        19.571429   

   HOME_PLAYER_ACCURATE_PASSES_season_sum_goalkeeper  \
0                                               13.0   
1                                               14.0   
2                                               14.5   
3                                                9.5   
4                                                6.0   

   HOME_PLAYER_ACCURATE_PASSES_season_sum_midfielder  \
0                                          16.545455   
1                                          11.500000   
2                                          28.625000   
3                                          29.300000   
4                                          20.600000   

   HOME_PLAYER_AERIALS_WON_season_sum_attacker  \
0                                     1.750000   
1                                    21.666667   
2                                    12.500000   
3                                          NaN   
4                                    18.250000   

   HOME_PLAYER_AERIALS_WON_season_sum_defender  \
0                                    16.000000   
1                                    24.000000   
2                                    22.750000   
3                                    28.333333   
4                                    18.142857   

   HOME_PLAYER_AERIALS_WON_season_sum_goalkeeper  \
0                                            1.0   
1                                            3.0   
2                                            0.5   
3                                            6.0   
4                                            0.5   

   HOME_PLAYER_AERIALS_WON_season_sum_midfielder  \
0                                       7.363636   
1                                       6.250000   
2                                       8.375000   
3                                      17.700000   
4                                      14.800000   

   HOME_PLAYER_ASSISTS_season_sum_attacker  \
0                 

In [23]:
train_data_players = pd.read_csv('../data/Train_Data/players_df.csv')

missing_columns_1 = find_missing_columns_bidirectional(train_data_players, sub_data_players)

# print("Colonnes présentes dans 1 mais manquantes dans 2 :", missing_columns['missing_in_df2'])
# print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns['missing_in_df1'])

sub_data_players = sub_data_players.drop(columns=missing_columns_1['missing_in_df1'])

missing_columns_home_2 = find_missing_columns_bidirectional(train_data_players, sub_data_players)

print("Colonnes présentes dans 2 mais manquantes dans 1 :", missing_columns_home_2['missing_in_df1'])

Colonnes présentes dans 2 mais manquantes dans 1 : []


In [24]:
spot_columns(sub_data_players, percent=10)

In [25]:
# On va tout input avec les médianes comme il n'y a pas de colonnes où il manque
# plus de 10% de NaNs. 

for col in sub_data_players.columns[sub_data_players.isna().mean() < 1]:
    sub_data_players.fillna({col: sub_data_players[col].median()}, inplace=True)

spot_columns(sub_data_players, percent=0)

In [26]:
sub_data_players.shape

(25368, 1081)

In [27]:
# Fusionner avec les données des équipes.
sub_data = sub_data_teams.merge(sub_data_players, on='ID', how='left')

sub_data = sub_data.set_index('ID')

sub_data.head(5)

HOME_TEAM_SHOTS_TOTAL_season_sum  HOME_TEAM_SHOTS_INSIDEBOX_season_sum  \
ID                                                                              
12303                               3.0                                   6.0   
12304                               3.0                                   2.0   
12305                               7.0                                  10.0   
12306                               5.0                                   7.0   
12307                               5.0                                   4.0   

       HOME_TEAM_SHOTS_OFF_TARGET_season_sum  \
ID                                             
12303                                    5.0   
12304                                    4.0   
12305                                    8.0   
12306                                    5.0   
12307                                    6.0   

       HOME_TEAM_SHOTS_ON_TARGET_season_sum  \
ID                                            
12303                                   1.0   
12304                                   2.0   
12305                                   8.0   
12306                                   9.0   
12307                                   3.0   

       HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum  HOME_TEAM_PASSES_season_sum  \
ID                                                                          
12303                                    3.0                          9.0   
12304                                    6.0                          1.0   
12305                                    1.0                          4.0   
12306                                    5.0                          7.0   
12307                                    5.0                          0.0   

       HOME_TEAM_SUCCESSFUL_PASSES_season_sum  HOME_TEAM_SAVES_season_sum  \
ID                                                                          
12303                                     8.0                        10.0   
12304                                     1.0                         2.0   
12305                                     5.0                         0.0   
12306                                     6.0                         3.0   
12307                                     0.0                         2.0   

       HOME_TEAM_CORNERS_season_sum  HOME_TEAM_FOULS_season_sum  \
ID                                                                
12303                           6.0                         7.0   
12304                           4.0                        10.0   
12305                          10.0                         4.0   
12306                           5.0                         8.0   
12307                           2.0                         9.0   

       HOME_TEAM_YELLOWCARDS_season_sum  HOME_TEAM_REDCARDS_season_sum  \
ID                                                                       
12303                               6.0                            3.0   
12304                               7.0                           10.0   
12305                               2.0                            2.0   
12306                               6.0                            8.0   
12307                              10.0                           10.0   

       HOME_TEAM_OFFSIDES_season_sum  HOME_TEAM_ATTACKS_season_sum  \
ID                                                                   
12303                            2.0                           7.0   
12304                            3.0                           2.0   
12305                            0.0                           6.0   
12306                            6.0                           2.0   
12307                            5.0                           8.0   

       HOME_TEAM_PENALTIES_season_sum  HOME_TEAM_SUBSTITUTIONS_season_sum  \
ID                                                                          
12303                             2.0                     

In [28]:
# Exportation du fichier final

sub_data.to_csv('../data/Sub_Data/5_sub_data.csv')